# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="deepseek-ai/DeepSeek-V3.1")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

c:\Users\labadmin\Desktop\AISWE\AG-AISOFTDEV\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ LLM Client configured: Using 'huggingface' with model 'deepseek-ai/DeepSeek-V3.1'


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [2]:
# TODO: Write a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
# Your prompt here
ROLE: you are a senior QA engineer.
TASK: write test for a FastAPI application using pytest.

The application code is below. Write tests that cover the main functionality of the application.
test 1 tests the user creation endpoint:
generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body)

test 2  tests the user retrieval endpoint:
test for the `GET /users/` endpoint

application code:
{app_code}

Save the generated tests into a file named `tests/test_main_simple.py`
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py")
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
# tests/test_main_simple.py
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import StaticPool

from main import app, get_db
from models import Base, User

# Configure test database
TEST_DATABASE_URL = "sqlite:///:memory:"
engine = create_engine(
    TEST_DATABASE_URL,
    connect_args={"check_same_thread": False},
    poolclass=StaticPool,
)
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Override get_db dependency
def override_get_db():
    try:
        db = TestingSessionLocal()
        yield db
    finally:
        db.close()

app.dependency_overrides[get_db] = override_get_db

# Test client
client = TestClient(app)

@pytest.fixture(scope="function")
def test_db():
    Base.metadata.create_all(bind=engine)
    yield
    Base.metadata.drop_all(bind=engine)

def test_create_user(test_db):
    """Test user cr

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [3]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
# Your prompt here
ROLE: you are a senior QA engineer.
TASK: write test for a FastAPI application using pytest.

The application code is below. Write tests that cover the main functionality of the application.
test 1 tests the user creation endpoint:
generate a `pytest` test function for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.

test 2  tests the user retrieval endpoint:
generate a `pytest` test function for the `GET /users/` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

application code:
{app_code}

Output only the raw Python code for these two test functions.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import StaticPool

# Import your FastAPI app
from main import app, Base, get_db

# Test database setup
TEST_DATABASE_URL = "sqlite:///:memory:"
engine = create_engine(
    TEST_DATABASE_URL,
    connect_args={"check_same_thread": False},
    poolclass=StaticPool
)
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Override the dependency
def override_get_db():
    try:
        db = TestingSessionLocal()
        yield db
    finally:
        db.close()

app.dependency_overrides[get_db] = override_get_db

# Test 1: Create user with duplicate email
def test_create_user_duplicate_email():
    # Create tables
    Base.metadata.create_all(bind=engine)
    
    client = TestClient(app)
    
    # First user creation
    user_data = {
        "email": "test@example.com",
  

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [4]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f""" You are a senior Python developer and QA engineer. Using the application source provided below (inserted as {app_code}), generate two Python files for pytest-based isolated DB testing:

conftest.py — a professional, well-documented pytest fixture implementation that:

Creates an in-memory SQLite engine: sqlite:///:memory: with connect_args={{"check_same_thread": False}}.
Instantiates a SQLAlchemy Session factory (e.g., SessionLocal = sessionmaker(...)).
Creates the DB schema before each test (or for the fixture scope used) and drops it afterward to guarantee a clean state.
Provides a fixture that yields a SQLAlchemy Session object and guarantees proper close/teardown.
Overrides the FastAPI app's get_db dependency to yield sessions from this test session factory while tests run.
Provides a client fixture that returns a fastapi.testclient.TestClient using the app with dependency overrides applied.
Uses explicit imports and references from the provided application code (use the app's app instance, DB Base/models, and original get_db name if present; adapt names if they differ in {app_code}).
Prefer scope="function" for isolation. If you choose scope="session" explain (in comments only) why and implement nested transaction rollback per-test.
Include type hints, concise docstrings, and clear comments for each fixture.
test_main_with_fixture.py — a refactor of the "happy-path" tests (Challenge 1) so they:

Use the client fixture from conftest.py.
Test the POST /users/ happy path asserting success status code and response body content.
Test the GET /users/ happy path asserting the list/created user is returned as expected.
Use only the test client; do not re-create sessions manually.
Keep tests deterministic and independent.
Output formatting requirements (STRICT):

Output only the raw Python contents for the two files and nothing else.
Precede each file's contents with a single-line filename marker comment exactly like:
=== FILE: conftest.py ===
<file contents here> # === FILE: [test_main_with_fixture.py](http://_vscodecontentref_/6) === <file contents here>
Do not print prose, explanations, or any additional markers beyond the two file markers and valid Python code.
Quality expectations:

Working, runnable pytest code that imports from the project's modules found in {app_code}.
Uses fastapi.testclient.TestClient and clean dependency override pattern: app.dependency_overrides[get_db] = override_get_db.
Proper session lifecycle: create_all before yield, drop_all after yield (or use nested transactions with rollback per test if you document that choice only as inline comments).
Clear, maintainable code style and helpful inline comments.
Application code context (insert below this line — use it to adapt names and imports): {app_code} """

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
# Your prompt here
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py")
else:
    print("Skipping test refactoring because app context is missing.")

--- Generating Pytest DB Fixture ---
We are going to create two files: conftest.py and test_main_with_fixture.py.
 The conftest.py will set up the test database and fixtures.
 The test_main_with_fixture.py will contain the tests using the fixtures.

 We note that the application code is provided in the same file (main.py) and we are going to import from it.

 However, note that the provided application code is in a single block. We must structure our test files to work with that.

 Since the application code defines:
   - Base (declarative base)
   - User and OnboardingTask models
   - app (FastAPI instance)
   - get_db dependency

 We will import these in our conftest.py and test_main_with_fixture.py.

 Steps for conftest.py:
   1. Create an in-memory SQLite database engine.
   2. Create a SessionLocal for tests.
   3. Create a fixture for the database session that creates the tables before and drops them after (or use transactions and rollback).
   4. Override the get_db dependency t

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.